In [1]:
# Import needed libraries
import folium
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import matplotlib.pyplot as plt
import contextily as ctx

In [2]:
# Filepaths
mapdata_fp = "data/maakunnat_2021_milj.shp"
info_fp = "data/info.txt"
for_tooltips_fp = "data/pop_data.geojson"

# Read files
mapdata = gpd.read_file(mapdata_fp)
info = pd.read_csv(info_fp, sep=';')
for_tooltips = gpd.read_file(for_tooltips_fp)

In [3]:
# Merge data sets
data = mapdata.merge(info, left_on='NAMEFIN', right_on='name')

In [4]:
# Re-project to WGS84
data = data.to_crs(epsg=4326)

In [5]:
data['pop_den'] = data['pop_den'].astype(float)

In [6]:
output_fp3 = 'data/pop_data.geojson'
data.to_file(output_fp3, driver='GeoJSON')

import mapclassify
# Plot figure
data['geoid'] = data.index.astype(str)

m = folium.Map(location=[65, 25], zoom_start=5)

bins = [1.9, 3.5, 11.3, 16.3, 23.8, 27.5, 32.8, 41.6, 45.2, 187.5]
folium.Choropleth(geo_data = data,
                  data = data,
                  columns=['geoid','pop_den'],
                  key_on='feature.id',
                  fill_color='OrRd',
                  line_color='black',
                  line_weight=1,
                  bins=bins,
                  legend_name= 'Population density in Finland (person per square kilometer)').add_to(m)

folium.features.GeoJson(for_tooltips,  
                        name='Labels',
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['NAMEFIN','pop_den'],
                                                                aliases = ['Region','Persons/km2'],
                                                                labels=True,
                                                                sticky=False,)
                                                               ).add_to(m)

outfp = "data/interactivemap.html"
m.save(outfp)